In [1]:
import numpy as np
import pandas as pd
import utils
import seaborn as sns
import matplotlib.pyplot as plt
import os, sys, glob
import itertools
%load_ext autoreload
%autoreload 2
import statsmodels.api as sm
from statsmodels.stats import multitest
from scipy.spatial.distance import cdist, pdist, squareform
from random import choices, choice
import nibabel as nib
from sklearn.preprocessing import MinMaxScaler
import corrstats
import scipy.stats
from scipy import stats
import phate, scprep
from sklearn.linear_model import LogisticRegression
import pingouin as pg
import matplotlib as mpl
import matplotlib
import run_exogenous_phate as ephate
from functools import reduce
from config import * 
from stats_functions import * 
from plot_utils import *
import mds
from sklearn.decomposition import PCA
import phate
np.random.seed(4)

SEED=4
np.random.seed(SEED)
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

sns.set(context='notebook', style='white')
plt.rcParams['legend.title_fontsize'] = 'xx-small'
plt.rcParams['legend.fontsize'] = 'xx-small'


(-0.8912799047605414, 0.3749185174912557)
(1.0171097212822702, 0.3091012362386929)


In [2]:
# Aggregate intermediate data
# # aggregate all the files
# longitudinal_df = pd.concat([pd.read_csv(f,index_col=0) for f in glob.glob(RESULTS_DIR_INTERMEDIATE+'/longitudinal*_v8.csv')]).reset_index(drop=True)
# baseline_df = pd.concat([pd.read_csv(f,index_col=0) for f in glob.glob(RESULTS_DIR_INTERMEDIATE+'/baseline*_v2.csv')]).reset_index(drop=True)

# # label what's got the embedding or not
# longitudinal_df['is_embed']=np.ones(len(longitudinal_df))
# for i in longitudinal_df.index:
#     if longitudinal_df.iloc[i]['embedding_type']=='brain':
#         longitudinal_df.at[i,'is_embed']=0

# baseline_df['is_embed']=np.ones(len(baseline_df))
# for i in baseline_df.index:
#     if baseline_df.iloc[i]['embedding_type']=='brain':
#         baseline_df.at[i,'is_embed']=0

# longitudinal_df.to_csv(f'{FINAL_RESULTS}/longitudinal_prediction_results.csv')
# baseline_df.to_csv(f'{FINAL_RESULTS}/cross_sectional_prediction_results.csv')

# # reload 
longitudinal_df = pd.read_csv(f'{FINAL_RESULTS}/longitudinal_prediction_results.csv',index_col=0)
baseline_df = pd.read_csv(f'{FINAL_RESULTS}/cross_sectional_prediction_results.csv',index_col=0)

In [3]:
main_methods = ['brain','vanilla_PHATE','EPHATE_5']
main_dvs = PREDICTORS_OF_INTEREST[5:8]
supp_methods_f3 = ['PCA','UMAP','EPHATE_5']
supp_methods_f4 = ['EPHATE_control','PHATE_PLUS_FEATURES','EPHATE_5']
supp_methods_f5 = ['EPHATE_ADI','EPHATE_SSCEY', 'EPHATE_5']
supp_DVs = PREDICTORS_OF_INTEREST[:5]
METHOD_SETS = {'main': main_methods, 
               'meth_s3':supp_methods_f3, 
               'meth_s4': supp_methods_f4, 
               'meth_s5':supp_methods_f5}
CONTRAST_LABELS = ['','']

# Figure 2

In [15]:
DVs = ['tfmri_nb_all_beh_ctotal_rate']
result_title = 'figure2'
methods_here = ['brain','vanilla_PHATE']
n=20
y='partial_corr_rho'
titles=['EN-Back task performance']
xvals_here={'brain_vanilla_PHATE': (-0.2, 0.2)}
yvals_here = {'brain_vanilla_PHATE':0.63}
hue_order=methods_here
dv = DVs[0]
# filter to get cross-validation, embedding types, and embeddings in 20 dimensions (or full voxel dim)
baseline_df_here = baseline_df[(baseline_df['fold']!='overall') & (baseline_df['embedding_type'].isin(methods_here))
                    &((baseline_df['n_dimensions']==n) | (baseline_df['is_embed']==0)) 
& (baseline_df['predictor']== dv)].reset_index(drop=True)

# run pairwise statistics, permutation test 2 tailed
stat_df_here = run_pairwise_stats(baseline_df_here, y, methods_here, nperm=10000, alt='two-sided')
# get bootstrapped CIs of mean
bs_resx = return_bootstrap_info(baseline_df_here)
bs_resx['event']='baseline'
bs_resx['DV']=dv
stat_df_here['event']='baseline'
stat_df_here['DV']=dv
# plot barplot with statistical significance
barplot_signif(baseline_df_here, stat_df_here, y, hue_order=hue_order, hue_label='embedding_type', 
                   ylabel=y, xname='contrast', 
                   columns='ROI_name', ylim=(-0.1, 0.7), yax=True, 
                   xvals=xvals_here, yvals=yvals_here, xlabels=CONTRAST_LABELS, title=f'{titles[0]}',
                   outfn=f'{FINAL_PLOTS}/{result_title}.pdf')
stat_df_here.to_csv(f'{FINAL_RESULTS}/{result_title}_pairwise_stats_cross_sectional.csv')
bs_resx.to_csv(f'{FINAL_RESULTS}/{result_title}_confidence_interval_cross_sectional.csv')


<Figure size 2662.25x500 with 0 Axes>

# Figure 3

In [16]:
DVs = main_dvs
methods_here = main_methods
baseline_df_here = baseline_df[(baseline_df['fold']!='overall') & (baseline_df['embedding_type'].isin(methods_here))].reset_index(drop=True)
titles=['Baseline CBCL total problem', 'Baseline CBCL internalizing', 'Baseline CBCL externalizing']

xvals_here = {'brain_vanilla_PHATE': (-0.25, -0.05), 'EPHATE_5_brain': (-0.25, 0.25), 'EPHATE_5_vanilla_PHATE':(0.05, 0.25)}
yvals_here = {'brain_vanilla_PHATE':0.19, 'EPHATE_5_vanilla_PHATE':0.19, 'EPHATE_5_brain': 0.21}

ylim=(-0.1, 0.23)
stats_df_list=[]
ci_list_here=[]
cols=['predictor']
y='partial_corr_rho'
hue_order=methods_here
n=20
result_title='figure3'
df1=baseline_df_here[(baseline_df_here['n_dimensions']==n) | (baseline_df_here['is_embed']==0)].reset_index(drop=True)
# run for each DV
for i in range(len(DVs)): 
    dv=DVs[i]
    result_title_i = f'{result_title}_{dv}'
    vals = [dv]
    df_here = extract_results(df1, cols, vals).reset_index(drop=True)
    stat_df_here = run_pairwise_stats(df_here, y, methods_here, nperm=10000, alt='two-sided')
    
    bs_resx = return_bootstrap_info(df_here)
    bs_resx['event']='baseline'
    bs_resx['DV']=dv
    bs_resx['n_dimensions']=n
    
    stat_df_here['event']='baseline'
    stat_df_here['DV']=dv
    stat_df_here['n_dimensions']=n
    
    stats_df_list.append(stat_df_here)
    ci_list_here.append(bs_resx)
    
    barplot_signif(df_here, stat_df_here, y, hue_order=hue_order, hue_label='embedding_type', 
                   ylabel=y, xname='contrast', 
                   columns='ROI_name', ylim=ylim, yax=True, 
                   xvals=xvals_here, yvals=yvals_here, xlabels=CONTRAST_LABELS, title=f'{titles[i]}',
                   outfn=f'{FINAL_PLOTS}/{result_title_i}.pdf')

stat_df=pd.concat(stats_df_list).reset_index(drop=True)
bs_resx=pd.concat(ci_list_here).reset_index(drop=True)
stat_df_here.to_csv(f'{FINAL_RESULTS}/{result_title}_pairwise_stats_cross_sectional.csv')
bs_resx.to_csv(f'{FINAL_RESULTS}/{result_title}_confidence_interval_cross_sectional.csv')

tfmri_nb_all_beh_ctotal_rate 20


<Figure size 2662.25x500 with 0 Axes>

<Figure size 2662.25x500 with 0 Axes>

<Figure size 2662.25x500 with 0 Axes>

# Figure 4

In [19]:
DVs = main_dvs
methods_here = main_methods
longitudinal_df = longitudinal_df[(longitudinal_df['fold']!='overall') & (longitudinal_df['embedding_type'].isin(methods_here))].reset_index(drop=True)
titles=['2-year CBCL total problem', '2-year CBCL internalizing', '2-year CBCL externalizing']

xvals_here = {'brain_vanilla_PHATE': (-0.25, -0.05), 'EPHATE_5_brain': (-0.25, 0.25), 'EPHATE_5_vanilla_PHATE':(0.05, 0.25)}
yvals_here={'brain_vanilla_PHATE': 0.12,
 'EPHATE_5_vanilla_PHATE': 0.12,
 'EPHATE_5_brain': 0.14}

ylim=(-0.1, 0.16)
stats_df_list=[]
ci_list_here=[]
cols=['predictor']
y='partial_corr_rho'
hue_order=methods_here
n=20
result_title='figure4'
df1=longitudinal_df[(longitudinal_df['n_dimensions']==n) | (longitudinal_df['is_embed']==0)].reset_index(drop=True)
stats_df_list, ci_list_here = [], []
# run for each DV
for i in range(len(DVs)): 
    dv=DVs[i]
    result_title_i = f'{result_title}_{dv}'
    vals = [dv]
    df_here = extract_results(df1, cols, vals).reset_index(drop=True)
    stat_df_here = run_pairwise_stats(df_here, y, methods_here, nperm=10000, alt='greater')
    
    bs_resx = return_bootstrap_info(df_here)
    bs_resx['event']='baseline'
    bs_resx['DV']=dv
    bs_resx['n_dimensions']=n
    
    stat_df_here['event']='baseline'
    stat_df_here['DV']=dv
    stat_df_here['n_dimensions']=n
    
    stats_df_list.append(stat_df_here)
    ci_list_here.append(bs_resx)
    
    barplot_signif(df_here, stat_df_here, y, hue_order=hue_order, hue_label='embedding_type', 
                   ylabel=y, xname='contrast', 
                   columns='ROI_name', ylim=ylim, yax=True, 
                   xvals=xvals_here, yvals=yvals_here, xlabels=CONTRAST_LABELS, title=f'{titles[i]}',
                   outfn=f'{FINAL_PLOTS}/{result_title_i}.pdf')
    print(result_title_i)
    

stat_df=pd.concat(stats_df_list).reset_index(drop=True)
bs_resx=pd.concat(ci_list_here).reset_index(drop=True)
stat_df_here.to_csv(f'{FINAL_RESULTS}/{result_title}_pairwise_stats_longitudinal.csv')
bs_resx.to_csv(f'{FINAL_RESULTS}/{result_title}_confidence_interval_longitudinal.csv')

figure4_cbcl_scr_syn_totprob_t
figure4_cbcl_scr_syn_internal_t
figure4_cbcl_scr_syn_external_t


<Figure size 2662.25x500 with 0 Axes>

<Figure size 2662.25x500 with 0 Axes>

<Figure size 2662.25x500 with 0 Axes>

# Figure S2

In [21]:
DVs = supp_DVs
methods_here = main_methods
baseline_df_here = baseline_df[(baseline_df['fold']!='overall') & (baseline_df['embedding_type'].isin(methods_here))].reset_index(drop=True)

titles=['CBCL anxdep', 'CBCL withdep', 'CBCL somatic', 'CBCL rulebreaking', 'CBCL aggressive']

xvals_here = {'brain_vanilla_PHATE': (-0.25, -0.05), 'EPHATE_5_brain': (-0.25, 0.25), 'EPHATE_5_vanilla_PHATE':(0.05, 0.25)}
yvals_here = {'brain_vanilla_PHATE':0.19, 'EPHATE_5_vanilla_PHATE':0.19, 'EPHATE_5_brain': 0.21}

ylim=(-0.1, 0.23)
stats_df_list=[]
ci_list_here=[]
cols=['predictor']
y='partial_corr_rho'
hue_order=methods_here
n=20
result_title='figure_s2'
df1=baseline_df_here[(baseline_df_here['n_dimensions']==n) | (baseline_df_here['is_embed']==0)].reset_index(drop=True)
# run for each DV
for i in range(len(DVs)): 
    dv=DVs[i]
    result_title_i = f'{result_title}_{dv}'
    vals = [dv]
    df_here = extract_results(df1, cols, vals).reset_index(drop=True)
    stat_df_here = run_pairwise_stats(df_here, y, methods_here, nperm=10000, alt='two-sided')
    
    bs_resx = return_bootstrap_info(df_here)
    bs_resx['event']='baseline'
    bs_resx['DV']=dv
    bs_resx['n_dimensions']=n
    
    stat_df_here['event']='baseline'
    stat_df_here['DV']=dv
    stat_df_here['n_dimensions']=n
    
    stats_df_list.append(stat_df_here)
    ci_list_here.append(bs_resx)
    
    barplot_signif(df_here, stat_df_here, y, hue_order=hue_order, hue_label='embedding_type', 
                   ylabel=y, xname='contrast', 
                   columns='ROI_name', ylim=ylim, yax=True, 
                   xvals=xvals_here, yvals=yvals_here, xlabels=CONTRAST_LABELS, title=f'{titles[i]}',
                   outfn=f'{FINAL_PLOTS}/{result_title_i}.pdf')
    print(result_title_i)

stat_df=pd.concat(stats_df_list).reset_index(drop=True)
bs_resx=pd.concat(ci_list_here).reset_index(drop=True)
stat_df_here.to_csv(f'{FINAL_RESULTS}/{result_title}_pairwise_stats_cross_sectional.csv')
bs_resx.to_csv(f'{FINAL_RESULTS}/{result_title}_confidence_interval_cross_sectional.csv')

figure_s2_cbcl_scr_syn_anxdep_t
figure_s2_cbcl_scr_syn_withdep_t
figure_s2_cbcl_scr_syn_somatic_t
figure_s2_cbcl_scr_syn_rulebreak_t
figure_s2_cbcl_scr_syn_aggressive_t


<Figure size 2662.25x500 with 0 Axes>

<Figure size 2662.25x500 with 0 Axes>

<Figure size 2662.25x500 with 0 Axes>

<Figure size 2662.25x500 with 0 Axes>

<Figure size 2662.25x500 with 0 Axes>

# Figure S3

In [25]:
DVs = main_dvs
methods_here = supp_methods_f3
baseline_df_here = baseline_df[(baseline_df['fold']!='overall') & (baseline_df['embedding_type'].isin(methods_here))].reset_index(drop=True)

titles=['CBCL total problem', 'CBCL internalizing', 'CBCL externalizing']
xvals_here = {'PCA_UMAP': (-0.25, -0.05), 'EPHATE_5_PCA': (-0.25, 0.25), 'EPHATE_5_UMAP':(0.05, 0.25)}
yvals_here = {'PCA_UMAP':0.19, 'EPHATE_5_UMAP':0.19, 'EPHATE_5_PCA': 0.21}

ylim=(-0.1, 0.23)
stats_df_list=[]
ci_list_here=[]
cols=['predictor']
y='partial_corr_rho'
hue_order=methods_here
n=20
result_title='figure_s3'
df1=baseline_df_here[(baseline_df_here['n_dimensions']==n) | (baseline_df_here['is_embed']==0)].reset_index(drop=True)
# run for each DV
for i in range(len(DVs)): 
    dv=DVs[i]
    result_title_i = f'{result_title}_{dv}'
    vals = [dv]
    df_here = extract_results(df1, cols, vals).reset_index(drop=True)
    stat_df_here = run_pairwise_stats(df_here, y, methods_here, nperm=10000, alt='two-sided')
    
    bs_resx = return_bootstrap_info(df_here)
    bs_resx['event']='baseline'
    bs_resx['DV']=dv
    bs_resx['n_dimensions']=n
    
    stat_df_here['event']='baseline'
    stat_df_here['DV']=dv
    stat_df_here['n_dimensions']=n
    
    stats_df_list.append(stat_df_here)
    ci_list_here.append(bs_resx)
    
    barplot_signif(df_here, stat_df_here, y, hue_order=hue_order, hue_label='embedding_type', 
                   ylabel=y, xname='contrast', 
                   columns='ROI_name', ylim=ylim, yax=True, 
                   xvals=xvals_here, yvals=yvals_here, xlabels=CONTRAST_LABELS, title=f'{titles[i]}',
                   outfn=f'{FINAL_PLOTS}/{result_title_i}.pdf')
    print(result_title_i)

stat_df=pd.concat(stats_df_list).reset_index(drop=True)
bs_resx=pd.concat(ci_list_here).reset_index(drop=True)
stat_df_here.to_csv(f'{FINAL_RESULTS}/{result_title}_pairwise_stats_cross_sectional.csv')
bs_resx.to_csv(f'{FINAL_RESULTS}/{result_title}_confidence_interval_cross_sectional.csv')

figure_s3_cbcl_scr_syn_totprob_t
figure_s3_cbcl_scr_syn_internal_t
figure_s3_cbcl_scr_syn_external_t


<Figure size 2652.1x500 with 0 Axes>

<Figure size 2652.1x500 with 0 Axes>

<Figure size 2652.1x500 with 0 Axes>

# Figure S4

In [21]:
DVs = main_dvs
methods_here = supp_methods_f4
baseline_df_here = baseline_df[(baseline_df['fold']!='overall') & (baseline_df['embedding_type'].isin(methods_here))].reset_index(drop=True)

titles=['CBCL total problem', 'CBCL internalizing', 'CBCL externalizing']

xvals_here = {'EPHATE_control_PHATE_PLUS_FEATURES': (-0.25, -0.05), 'EPHATE_5_EPHATE_control': (-0.25, 0.25), 'EPHATE_5_PHATE_PLUS_FEATURES':(0.05, 0.25)}
yvals_here = {'EPHATE_control_PHATE_PLUS_FEATURES':0.19, 'EPHATE_5_EPHATE_control':0.21, 'EPHATE_5_PHATE_PLUS_FEATURES': 0.19}
ylim=(-0.1, 0.23)
stats_df_list=[]
ci_list_here=[]
cols=['predictor']
y='partial_corr_rho'
hue_order=methods_here
n=20
result_title='figure_s4'
df1=baseline_df_here[(baseline_df_here['n_dimensions']==n) | (baseline_df_here['is_embed']==0)].reset_index(drop=True)
# run for each DV
for i in range(len(DVs)): 
    dv=DVs[i]
    result_title_i = f'{result_title}_{dv}'
    vals = [dv]
    df_here = extract_results(df1, cols, vals).reset_index(drop=True)
    stat_df_here = run_pairwise_stats(df_here, y, methods_here, nperm=10000, alt='two-sided')
    
    bs_resx = return_bootstrap_info(df_here)
    bs_resx['event']='baseline'
    bs_resx['DV']=dv
    bs_resx['n_dimensions']=n
    
    stat_df_here['event']='baseline'
    stat_df_here['DV']=dv
    stat_df_here['n_dimensions']=n
    
    stats_df_list.append(stat_df_here)
    ci_list_here.append(bs_resx)
    
    barplot_signif_patterns(df_here, stat_df_here, y, hue_order=hue_order, hue_label='embedding_type', 
                   ylabel=y, xname='contrast', 
                   columns='ROI_name', ylim=ylim, yax=True, 
                   xvals=xvals_here, yvals=yvals_here, xlabels=CONTRAST_LABELS, title=f'{titles[i]}',
                   outfn=f'{FINAL_PLOTS}/{result_title_i}.pdf')
    print(result_title_i)

stat_df=pd.concat(stats_df_list).reset_index(drop=True)
bs_resx=pd.concat(ci_list_here).reset_index(drop=True)
stat_df_here.to_csv(f'{FINAL_RESULTS}/{result_title}_pairwise_stats_cross_sectional.csv')
bs_resx.to_csv(f'{FINAL_RESULTS}/{result_title}_confidence_interval_cross_sectional.csv')

[0, 0, 2] {'nBack_emotion_vs_neutface': ['#FFFFFF', '#FFFFFF', '#ED5F68'], 'nBack_2_back_vs_0_back': ['#FFFFFF', '#FFFFFF', '#0F5257']} {'nBack_emotion_vs_neutface': ['\\', '+', ''], 'nBack_2_back_vs_0_back': ['\\', '+', '']}
figure_s4_cbcl_scr_syn_totprob_t
[0, 0, 2] {'nBack_emotion_vs_neutface': ['#FFFFFF', '#FFFFFF', '#ED5F68'], 'nBack_2_back_vs_0_back': ['#FFFFFF', '#FFFFFF', '#0F5257']} {'nBack_emotion_vs_neutface': ['\\', '+', ''], 'nBack_2_back_vs_0_back': ['\\', '+', '']}
figure_s4_cbcl_scr_syn_internal_t
[0, 0, 2] {'nBack_emotion_vs_neutface': ['#FFFFFF', '#FFFFFF', '#ED5F68'], 'nBack_2_back_vs_0_back': ['#FFFFFF', '#FFFFFF', '#0F5257']} {'nBack_emotion_vs_neutface': ['\\', '+', ''], 'nBack_2_back_vs_0_back': ['\\', '+', '']}
figure_s4_cbcl_scr_syn_external_t


<Figure size 2735.75x500 with 0 Axes>

<Figure size 2735.75x500 with 0 Axes>

<Figure size 2735.75x500 with 0 Axes>

# Figure S5

In [20]:
DVs = main_dvs
methods_here = supp_methods_f5
baseline_df_here = baseline_df[baseline_df['embedding_type'].isin(supp_methods_f5)].reset_index(drop=True)
df1 = baseline_df_here[(baseline_df_here['fold']!='overall') & (baseline_df_here['n_dimensions']==n)].reset_index(drop=True)
titles=['CBCL total problem', 'CBCL internalizing', 'CBCL externalizing']

xvals_here = {'EPHATE_ADI_EPHATE_SSCEY': (-0.25, -0.05), 'EPHATE_5_EPHATE_ADI': (-0.25, 0.25), 'EPHATE_5_EPHATE_SSCEY':(0.05, 0.25)}
yvals_here = {'EPHATE_ADI_EPHATE_SSCEY':0.19, 'EPHATE_5_EPHATE_SSCEY':0.19, 'EPHATE_5_EPHATE_ADI': 0.21}

ylim=(-0.1, 0.23)
stats_df_list=[]
ci_list_here=[]
cols=['predictor']
y='partial_corr_rho'
hue_order=supp_methods_f5
n=20
result_title='figure_s5'
# run for each DV
for i in range(len(DVs)): 
    dv=DVs[i]
    result_title_i = f'{result_title}_{dv}'
    vals = [dv]
    df_here = extract_results(df1, cols, vals).reset_index(drop=True)
    stat_df_here = run_pairwise_stats(df_here, y, methods_here, nperm=10000, alt='two-sided')
    
    bs_resx = return_bootstrap_info(df_here)
    bs_resx['event']='baseline'
    bs_resx['DV']=dv
    bs_resx['n_dimensions']=n
    
    stat_df_here['event']='baseline'
    stat_df_here['DV']=dv
    stat_df_here['n_dimensions']=n
    
    stats_df_list.append(stat_df_here)
    ci_list_here.append(bs_resx)
    barplot_signif_patterns(df_here, stat_df_here, y, hue_order=hue_order, hue_label='embedding_type', 
                   ylabel=y, xname='contrast', 
                   columns='ROI_name', ylim=ylim, yax=True, 
                   xvals=xvals_here, yvals=yvals_here, xlabels=CONTRAST_LABELS, title=f'{titles[i]}',
                   outfn=f'{FINAL_PLOTS}/{result_title_i}.pdf')
    print(result_title_i)

stat_df=pd.concat(stats_df_list).reset_index(drop=True)
bs_resx=pd.concat(ci_list_here).reset_index(drop=True)
stat_df_here.to_csv(f'{FINAL_RESULTS}/{result_title}_pairwise_stats_cross_sectional.csv')
bs_resx.to_csv(f'{FINAL_RESULTS}/{result_title}_confidence_interval_cross_sectional.csv')

[3, 4, 2] {'nBack_emotion_vs_neutface': ['#FFFFFF', '#FFFFFF', '#ED5F68'], 'nBack_2_back_vs_0_back': ['#FFFFFF', '#FFFFFF', '#0F5257']} {'nBack_emotion_vs_neutface': ['oo', '--', ''], 'nBack_2_back_vs_0_back': ['oo', '--', '']}
figure_s5_cbcl_scr_syn_totprob_t
[3, 4, 2] {'nBack_emotion_vs_neutface': ['#FFFFFF', '#FFFFFF', '#ED5F68'], 'nBack_2_back_vs_0_back': ['#FFFFFF', '#FFFFFF', '#0F5257']} {'nBack_emotion_vs_neutface': ['oo', '--', ''], 'nBack_2_back_vs_0_back': ['oo', '--', '']}
figure_s5_cbcl_scr_syn_internal_t
[3, 4, 2] {'nBack_emotion_vs_neutface': ['#FFFFFF', '#FFFFFF', '#ED5F68'], 'nBack_2_back_vs_0_back': ['#FFFFFF', '#FFFFFF', '#0F5257']} {'nBack_emotion_vs_neutface': ['oo', '--', ''], 'nBack_2_back_vs_0_back': ['oo', '--', '']}
figure_s5_cbcl_scr_syn_external_t


<Figure size 2670.5x500 with 0 Axes>

<Figure size 2670.5x500 with 0 Axes>

<Figure size 2670.5x500 with 0 Axes>

In [36]:
# all these figures are cross-sectional
all_stats=[]
all_ci = []
drop = ['r2z_p','n_per_fold','sorted_comparison','yname','perm_p','corrected_p','symbol']
for result_title in ['figure2','figure3','figure4','figure_s2','figure_s3','figure_s4','figure_s5']:
    stats_info = pd.read_csv(glob.glob(f'{FINAL_RESULTS}/{result_title}_pairwise_stats_*.csv')[0],index_col=0)
    ci_info = pd.read_csv(glob.glob(f'{FINAL_RESULTS}/{result_title}_confidence_interval_*.csv')[0], index_col=0)
    if result_title == 'figure_4':
        event='2year'
    else:
        event='baseline'
    stats_info['event']=np.repeat(event,len(stats_info))
    stats_info.drop(drop, axis=1, inplace=True) # drop unnecessary/redundant info
    ci_info['event']=np.repeat(event,len(ci_info))
    try:
        ci_info.drop(['predictor'],axis=1,inplace=True)
    except:
        continue
    
    all_stats.append(stats_info)
    all_ci.append(ci_info)
pd.concat(all_ci).reset_index(drop=True).to_csv(f'{FINAL_RESULTS}/data_table1.csv')
pd.concat(all_stats).reset_index(drop=True).to_csv(f'{FINAL_RESULTS}/data_table2.csv')

